In [5]:
from Model.DL_Network_Model import Net
from Model.Funtion_Bank import data_input
import os.path
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile
from PIL import *
import PIL.Image
import random
import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
import random
from shutil import copyfile
from tkinter import *
global btn_txt_matrix
global btn_matrix
global btn_refresh_matrix
global Button
global Current_play_ground
import time

input_file_path = 'Data/training_data_input.csv'
score_file_path = 'Data/training_data_score.csv'
next_action_file_path = 'Data/training_data_next_action_taken.csv'
#input_file_path_initiate = 'Data/training_data_input_initiate.csv'
#score_file_path_initiate = 'Data/training_data_score_initiate.csv'
training_file_path = 'Data/training_data.csv'

DQ_ratio = 0.9
file_name_model_latest_version = 'Model/model_latest_version.pt'
file_name_model_last_version = 'Model/model_last_version.pt'

#data format setup:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)

def play_ground(h, w, agent_a, agent_b, agent_c, agent_d, ball):
    rows = h
    columns = w * 3 + 2
    init_ground = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.float)
    for i in range(0, rows):
        for j in [w, -w - 1]:
            init_ground[i][j] = 1
    for i in [agent_a[0]]:
        for j in range(agent_a[1] + w, agent_a[3] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_b[0]]:
        for j in range(agent_b[1] + w, agent_b[3] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_c[0]]:
        for j in range(agent_c[3] + w, agent_c[1] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_d[0]]:
        for j in range(agent_d[3] + w, agent_d[1] + 1 + w):
            init_ground[i][j] = 1
    init_ground[ball[0]][ball[1]] = 2
    return init_ground

def agent_ball(perception, state):
    rows = 8
    columns = 3
    reference_matrix = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.int)
    reference_matrix[0][1] = -1
    reference_matrix[0][2] = -1
    reference_matrix[1][1] = -1
    reference_matrix[1][2] = 0
    reference_matrix[2][1] = -1
    reference_matrix[2][2] = 1
    reference_matrix[3][1] = 0
    reference_matrix[3][2] = 1
    reference_matrix[4][1] = 1
    reference_matrix[4][2] = 1
    reference_matrix[5][1] = 1
    reference_matrix[5][2] = 0
    reference_matrix[6][1] = 1
    reference_matrix[6][2] = -1
    reference_matrix[7][1] = 0
    reference_matrix[7][2] = -1

    if (state[2] * state[3]) == 0:
        if perception == [1, 1, 1]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception == [0, 1, 1]:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
                    print(perception)
                    print(start_point)
            new_speed_x = reference_matrix[start_point - 2][1]
            new_speed_y = reference_matrix[start_point - 2][2]
        elif perception == [1, 1, 0]:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
            new_speed_x = reference_matrix[(start_point + 2)% 8][1]
            new_speed_y = reference_matrix[(start_point + 2)% 8][2]
        elif perception == [0, 1, 0]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        else:
            new_speed_x = state[2]
            new_speed_y = state[3]
    else:
        if (perception[0] == 1) & (perception[2] == 1):
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception == [0, 1, 0]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception[2] == 1:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
                    print(perception)
                    print(start_point)
            new_speed_x = reference_matrix[start_point - 2][1]
            new_speed_y = reference_matrix[start_point - 2][2]
        elif perception[0] == 1:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
            new_speed_x = reference_matrix[(start_point + 2)% 8][1]
            new_speed_y = reference_matrix[(start_point + 2)% 8][2]
        else:
            new_speed_x = state[2]
            new_speed_y = state[3]

    action = [new_speed_x, new_speed_y]
    state = [state[0] + new_speed_x, state[1] + new_speed_y, new_speed_x, new_speed_y]
    return (action, state)


def func_perspection(play_ground, state):
    rows = 8
    columns = 3
    reference_matrix = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.int)
    reference_matrix[0][1] = -1
    reference_matrix[0][2] = -1
    reference_matrix[1][1] = -1
    reference_matrix[1][2] = 0
    reference_matrix[2][1] = -1
    reference_matrix[2][2] = 1
    reference_matrix[3][1] = 0
    reference_matrix[3][2] = 1
    reference_matrix[4][1] = 1
    reference_matrix[4][2] = 1
    reference_matrix[5][1] = 1
    reference_matrix[5][2] = 0
    reference_matrix[6][1] = 1
    reference_matrix[6][2] = -1
    reference_matrix[7][1] = 0
    reference_matrix[7][2] = -1
    for i in range(0,8):
        reference_matrix[i][0] = play_ground[state[0] + reference_matrix[i][1]][state[1] + reference_matrix[i][2]]
        if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
            start_point = i
    #print(reference_matrix)
    if (state[2] * state[3]) == 0:
        #print(state[2])
        #print(state[3])
        new_perception = [reference_matrix[start_point - 2][0], reference_matrix[start_point][0], reference_matrix[(start_point + 2)% 8][0]]
    else:
        new_perception = [reference_matrix[start_point - 1][0], reference_matrix[start_point][0], reference_matrix[(start_point + 1)% 8][0]]

    #print(new_perception)
    return new_perception


def DL_training_model():
    indicator = 0
    learning_rate = 0.1
    epoch_size = 10000
    steps_for_printing_out_loss = 1000
    #cropped_perspective = np.array([4,5,6])
    input_data = data_input(input_file_path)
    score_data = data_input(score_file_path)
    input = input_data
    #print(input)
    target = score_data
    net = Net()
    loss_functioin = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = learning_rate)
    if indicator == 1:
        state_dict_last_version = torch.load('Model/model_latest_version.pt')['state_dict']
        net.load_state_dict(state_dict_last_version)
    for i in range(1,epoch_size + 1):
        optimizer.zero_grad()
        output = net(input)
        output[target == -2] = -2
        #output[target == -1.1] = -1.1
        loss = loss_functioin(output, target)
        loss.backward()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
        # Does the update
        optimizer.step()
    
    if indicator == 1:
        copyfile(file_name_model_latest_version, file_name_model_last_version)
    else:
        torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_last_version)
    torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)


def DL_model(cropped_perspective, model_version, state):
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    next_vision = model_latest_version(torch.tensor(cropped_perspective.reshape(252), dtype=torch.float))
    #print(next_vision)
    if ((state[0] == -1) and (state[1] == 1)) or ((state[0] == 0) and (state[1] == playground_w)):
        next_vision[0] = -3
    if ((state[0] == -1) and (state[3] == playground_w)) or ((state[0] == 0) and (state[3] == 1)):
        next_vision[2] = -3
    #print('abc')
    #print(next_vision)
    next_step = next_vision.argmax()
    action = next_step
    return action

def game_agent(play_ground_matrix, state):
    result = 0
    if (2 in play_ground_matrix[0]):
        result = 'a'
    if (2 in play_ground_matrix[-1]):
        result = 'b'
    return result



def cropping_perspective(perception, state):
    mid_point = (state[1] + state[3]) / 2 + playground_w + 1
    if state[0] == -1:
        cropped_perspective = perception[:,int(mid_point - playground_w): int(mid_point + playground_w + 1)]
        cropped_perspective = cropped_perspective.cpu().numpy()
    else:
        cropped_perspective = perception[:,int(mid_point - playground_w): int(mid_point + playground_w + 1)]
        #print(int(mid_point - playground_w))
        #print(int(mid_point + playground_w))
        #cropped_perspective = perception[:][2:4]
        cropped_perspectiven_numpy = cropped_perspective.clone()
        cropped_perspective = np.flip(cropped_perspectiven_numpy.cpu().numpy(), 0)
        #print(cropped_perspective)
        #cropped_perspective = torch.tensor(cropped_perspective, dtype=torch.float)
    #print(cropped_perspective)
    cropped_perspective = torch.tensor(cropped_perspective.reshape(252), dtype=torch.float)
    return cropped_perspective
    
def agent_a(cropped_perspective, state):
    action = DL_model(cropped_perspective, 1, state) #-1, 1, 0
    #state = [-1, 1, -1, 3]
    if state[0] == 0:
        state[1] = state[1] - (action - 1)
        state[3] = state[3] - (action - 1)
    else:
        state[1] = state[1] + (action - 1)
        state[3] = state[3] + (action - 1)
    return (action, state)




def start_play_game():
    global Current_play_ground
    global export_output
    playground_h = 12
    playground_w = 10
    #print(play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], [3, 3]))
    #New_play_ground = play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], [3, 3])
    #print(agent_ball([1, 0, 0], [5, 15, 1, 1]))
    for rounds in range(0, 1):

        current_state = [9, 16, 1, 1]
        agent_a1_state = [-1, 1, -1, 3]
        agent_a2_state = [-1, 4, -1, 6]
        agent_b1_state = [0, 6, 0, 4]
        agent_b2_state = [0, 3, 0, 1]

        random_a1 = random.randint(1,6)
        random_a2 = random.randint(1,6)
        random_b1 = random.randint(1,6)
        random_b2 = random.randint(1,6)

        current_state = [random.randint(3,9), random.randint(12,19), random.choice([1, 0, -1]), random.choice([1, 0, -1])]
        if (current_state[2] * current_state[3]) == 0:
            current_state[2] = 1
            current_state[3] = -1
        agent_a1_state = [-1, 1 + random_a1, -1, 3 + random_a1]
        agent_a2_state = [-1, 1 + random_a2, -1, 3 + random_a2]
        agent_b1_state = [0, 10 - random_b1, 0, 8 - random_b1]
        agent_b2_state = [0, 10 - random_b2, 0, 8 - random_b2]

        #New_action, New_state = agent_ball([0, 0, 0], [5, 16, 1, 1])

        all_cropped_perspective_agent_a1 = []
        all_action_agent_a1 = []
        all_score_agent_a1 = []

        all_cropped_perspective_agent_a2 = []
        all_action_agent_a2 = []
        all_score_agent_a2 = []

        all_cropped_perspective_agent_b1 = []
        all_action_agent_b1 = []
        all_score_agent_b1 = []

        all_cropped_perspective_agent_b2 = []
        all_action_agent_b2 = []
        all_score_agent_b2 = []

        score_agent_a1 = 0
        score_agent_a2 = 0
        score_agent_b1 = 0
        score_agent_b2 = 0

        for i in range(0,30):
            Current_play_ground = play_ground(12,10,agent_a1_state, agent_a2_state, agent_b1_state, agent_b2_state, current_state[0: 2])
            #print(Current_play_ground)
            #print(game_agent(Current_play_ground, [1]))
            Current_perception = func_perspection(Current_play_ground, current_state)

            New_action, New_state = agent_ball(Current_perception, current_state)

            Current_play_ground[current_state[0]][current_state[1]] = -2
            current_state = New_state
            Current_play_ground[current_state[0]][current_state[1]] = 2

            #print(agent_b2_state)
            cropped_perspective_agent_a1 = cropping_perspective(Current_play_ground, agent_a1_state)
            cropped_perspective_agent_a2 = cropping_perspective(Current_play_ground, agent_a2_state)
            cropped_perspective_agent_b1 = cropping_perspective(Current_play_ground, agent_b1_state)
            cropped_perspective_agent_b2 = cropping_perspective(Current_play_ground, agent_b2_state)


            agent_a1_action, agent_a1_state = agent_a(cropped_perspective_agent_a1, agent_a1_state)
            agent_a2_action, agent_a2_state = agent_a(cropped_perspective_agent_a2, agent_a2_state)
            agent_b1_action, agent_b1_state = agent_a(cropped_perspective_agent_b1, agent_b1_state)
            agent_b2_action, agent_b2_state = agent_a(cropped_perspective_agent_b2, agent_b2_state)

            current_score = torch.ones(3) * -2

            all_cropped_perspective_agent_a1.append(cropped_perspective_agent_a1.cpu().numpy())
            all_action_agent_a1.append(agent_a1_action.cpu().numpy())
            all_score_agent_a1.append(current_score.cpu().numpy()) 

            all_cropped_perspective_agent_a2.append(cropped_perspective_agent_a2.cpu().numpy())
            all_action_agent_a2.append(agent_a2_action.cpu().numpy())
            all_score_agent_a2.append(current_score.cpu().numpy()) 

            all_cropped_perspective_agent_b1.append(cropped_perspective_agent_b1.cpu().numpy())
            all_action_agent_b1.append(agent_b1_action.cpu().numpy())
            all_score_agent_b1.append(current_score.cpu().numpy()) 

            all_cropped_perspective_agent_b2.append(cropped_perspective_agent_b2.cpu().numpy())
            all_action_agent_b2.append(agent_b2_action.cpu().numpy())
            all_score_agent_b2.append(current_score.cpu().numpy()) 


            if game_agent(Current_play_ground, [1]) == 'a':
                score_agent_a1 = 1
                score_agent_a2 = 1
                score_agent_b1 = -1
                score_agent_b2 = -1
                break
            elif game_agent(Current_play_ground, [1]) == 'b':
                score_agent_a1 = -1
                score_agent_a2 = -1
                score_agent_b1 = 1
                score_agent_b2 = 1
                break

            #print(all_cropped_perspective_agent_a1)
            #print(all_action_agent_a1)
            Current_play_ground = play_ground(12,10,agent_a1_state, agent_a2_state, agent_b1_state, agent_b2_state, current_state[0: 2])
            
            export_output = Current_play_ground.clone()
            export_output = export_output.cpu().detach().numpy()
            
#####################################
            refresh_game_board(export_output)
            print(i)
            #time.sleep(1)
            """
            export_output = export_output * 120
            img = Image.fromarray(export_output)
            #img.save('my.png')
            img.show()
            """
            #print(New_action)
            #print(New_state)
            #print(new_perception)
        #print(agent_a(Current_play_ground, [-1, 1, -1, 3]))
        for k in range(0, len(all_action_agent_a1)):
            all_score_agent_a1[-(k+1)][all_action_agent_a1[-(k+1)]] = ((DQ_ratio)**(k)) * score_agent_a1
            all_score_agent_a2[-(k+1)][all_action_agent_a2[-(k+1)]] = ((DQ_ratio)**(k)) * score_agent_a2
            all_score_agent_b1[-(k+1)][all_action_agent_b1[-(k+1)]] = ((DQ_ratio)**(k)) * score_agent_b1
            all_score_agent_b2[-(k+1)][all_action_agent_b2[-(k+1)]] = ((DQ_ratio)**(k)) * score_agent_b2
        #print(all_score_agent_a1)
        #print(all_score_agent_a2)
        #print(all_score_agent_b1)
        #print(all_score_agent_b2)

        input_status = all_cropped_perspective_agent_a1 + all_cropped_perspective_agent_a2 + all_cropped_perspective_agent_b1 + all_cropped_perspective_agent_b2
        next_action_taken = all_action_agent_a1 + all_action_agent_a2 + all_action_agent_b1 + all_action_agent_b2
        score = all_score_agent_a1 + all_score_agent_a2 + all_score_agent_b1 + all_score_agent_b2

        input_status_df = pd.DataFrame(input_status)
        input_status_df.to_csv(input_file_path, index=False, mode='a', header=False)
        next_action_taken_df = pd.DataFrame(next_action_taken)
        next_action_taken_df.to_csv(next_action_file_path, index=False, mode='a', header=False)
        score_df = pd.DataFrame(score)
        score_df.to_csv(score_file_path, index=False, mode='a', header=False)

    """
    #remove duplication:
    input_status_df = pd.read_csv(input_file_path, header = None)
    next_action_taken_df = pd.read_csv(next_action_file_path, header = None)
    score_df = pd.read_csv(score_file_path, header = None)
    consul_df = pd.concat([input_status_df, next_action_taken_df, score_df], axis=1)
    consul_df = consul_df.replace(-0.0, 0.0)
    consul_df.drop_duplicates(keep = 'first', inplace = True) 
    input_status_df = consul_df.iloc[:,0:252].copy()
    next_action_taken_df = consul_df.iloc[:,252:253].copy()
    score_df = consul_df.iloc[:,253:].copy()
    input_status_df.to_csv(input_file_path, index=False, header=False)
    next_action_taken_df.to_csv(next_action_file_path, index=False, header=False)
    score_df.to_csv(score_file_path, index=False, header=False)
    """

def refresh_game_board(export_output):
    global btn_txt_matrix
    data_original = export_output.reshape(384)
    for i in range(0,384):
        if data_original[i] == 1:
            btn_txt_matrix[i].set('I')
        elif data_original[i] == 2:
            btn_txt_matrix[i].set('o')
        else:
            btn_txt_matrix[i].set('')

def clock():
    time = datetime.datetime.now().strftime("Time: %H:%M:%S")
    lab.config(text=time)
    #lab['text'] = time
    root.after(1000, clock)
            
            
def game_UI():
    global btn_txt_matrix
    global btn_matrix
    global btn_refresh_matrix
    playground_h = 12
    playground_w = 10
    rows = playground_h
    columns = playground_w * 3 + 2
    btn_matrix = []
    btn_refresh_matrix = []
    btn_txt_matrix = []
    
    #load mode
    
    #Load game UI:
    window = Tk()
    window.title("Welcome to Game made by Hongxu")
    window.geometry('1400x600')
    lbl = Label(window, text="Ready to start")
    
    count = 0
    for i in range(0, rows):
        for j in range(0, columns):
            btn_txt_matrix.append(StringVar())
            #btn_refresh_matrix.append(button_refresh())
            btn_matrix.append(Button(window, textvariable = btn_txt_matrix[count], command = button_refresh(count), height = 2, width = 4))
            btn_matrix[count].grid(row = i, column = j)
            
            count += 1
    window.mainloop()
def button_refresh():
    global Current_play_ground
    global export_output
    print('o')
    start_play_game()
    
    
    
    
    """
    playground_h = 12
    playground_w = 10
    rows = playground_h
    columns = playground_w * 3 + 2
    #lbl.configure(text="Button was clicked !!")
    if btn_txt_matrix[0].get() == '':
        btn_txt_matrix[0].set('o')
    else:
        btn_txt_matrix[0].set('')
    print('o');

    
    for m in range(0, rows):
        for n in range(0, columns):
            #btn_txt_matrix.append(StringVar())
            btn_txt_matrix[count_no].set(str(m + n))
            count_no += 1

    if btn_20_text.get() == '':
        btn_20_text.set("o")
        data_original[6] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)
    """
    
if __name__ == "__main__":
    
    #Load game UI:
    window = Tk()
    window.title("Welcome to Game made by Hongxu")
    window.geometry('1400x600')
    lbl = Label(window, text="Ready to start")

    playground_h = 12
    playground_w = 10
    rows = playground_h
    columns = playground_w * 3 + 2
    btn_matrix = []
    btn_refresh_matrix = []
    btn_txt_matrix = []
    
    #load mode
    
    count = 0
    for i in range(0, rows):
        for j in range(0, columns):
            btn_txt_matrix.append(StringVar())
            
            #btn_refresh_matrix.append(button_refresh())
            btn_matrix.append(Button(window, textvariable = btn_txt_matrix[count], height = 2, width = 4))
            btn_txt_matrix[count].set('x')
            btn_matrix[count].grid(row = i, column = j)
            count += 1
    start_button = Button(window, text = 'S', command = button_refresh, height = 2, width = 4)
    start_button.grid(row = 14, column = 1)
    
    window.mainloop()
    
    
    
    
    k = 0
    if k == 0:
        print('s')
        #game_UI()
        #start_play_game()
    else:
        DL_training_model()



"""
#print(New_play_ground)
export_output = Current_play_ground.clone()
export_output = export_output.cpu().detach().numpy()
export_output = export_output * 120
img = Image.fromarray(export_output)
#img.save('my.png')
img.show()
#print(func_perspection(New_play_ground, New_state))
"""



o


D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:220: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0
1
2
3
4
5
6
7
8
9
10
11
12
o
0
1
2
3
4
5
o
0
o
0
1
2
3
4
[tensor(0, dtype=torch.int32), tensor(1, dtype=torch.int32), tensor(1, dtype=torch.int32)]
2
5
6
7
8
9
s


"\n#print(New_play_ground)\nexport_output = Current_play_ground.clone()\nexport_output = export_output.cpu().detach().numpy()\nexport_output = export_output * 120\nimg = Image.fromarray(export_output)\n#img.save('my.png')\nimg.show()\n#print(func_perspection(New_play_ground, New_state))\n"

In [26]:
#https://pastebin.com/mQrhpcEk

from tkinter import *
 
root = Tk()
 
files = [] #creates list to replace your actual inputs for troubleshooting purposes
btn = [] #creates list to store the buttons ins
 
for i in range(50): #this just popultes a list as a replacement for your actual inputs for troubleshooting purposes
    files.append("Button"+str(i))
    
for i in range(len(files)): #this says for *counter* in *however many elements there are in the list files*
    #the below line creates a button and stores it in an array we can call later, it will print the value of it's own text by referencing itself from the list that the buttons are stored in
    btn.append(Button(root, text=files[i], command=lambda c=i: onButtonClicked(c)))
    btn[i].pack() #this packs the buttons
    
def onButtonClicked(button_id):
    print("Button" + str(button_id) + " is clicked!");
    
root.mainloop()

Button27 is clicked!
Button24 is clicked!


In [16]:
"""
crop Matrix for 4 Agents
action: 1,2,3, array
RL model


draft: 3 output: score: 1, 0, -1???
draft: DL model

completed: save and load 3D data into local file (mat file)

"""

import numpy as np
import scipy.io

# Some test data
x = np.arange(200).reshape((4,5,10))

# Specify the filename of the .mat file
matfile = 'test_mat.mat'

# Write the array to the mat file. For this to work, the array must be the value
# corresponding to a key name of your choice in a dictionary
scipy.io.savemat(matfile, mdict={'out': x}, oned_as='row')

# For the above line, I specified the kwarg oned_as since python (2.7 with 
# numpy 1.6.1) throws a FutureWarning.  Here, this isn't really necessary 
# since oned_as is a kwarg for dealing with 1-D arrays.

# Now load in the data from the .mat that was just saved
matdata = scipy.io.loadmat(matfile)

# And just to check if the data is the same:
assert np.all(x == matdata['out'])

print(matdata['out'])

[[[  0   1   2   3   4   5   6   7   8   9]
  [ 10  11  12  13  14  15  16  17  18  19]
  [ 20  21  22  23  24  25  26  27  28  29]
  [ 30  31  32  33  34  35  36  37  38  39]
  [ 40  41  42  43  44  45  46  47  48  49]]

 [[ 50  51  52  53  54  55  56  57  58  59]
  [ 60  61  62  63  64  65  66  67  68  69]
  [ 70  71  72  73  74  75  76  77  78  79]
  [ 80  81  82  83  84  85  86  87  88  89]
  [ 90  91  92  93  94  95  96  97  98  99]]

 [[100 101 102 103 104 105 106 107 108 109]
  [110 111 112 113 114 115 116 117 118 119]
  [120 121 122 123 124 125 126 127 128 129]
  [130 131 132 133 134 135 136 137 138 139]
  [140 141 142 143 144 145 146 147 148 149]]

 [[150 151 152 153 154 155 156 157 158 159]
  [160 161 162 163 164 165 166 167 168 169]
  [170 171 172 173 174 175 176 177 178 179]
  [180 181 182 183 184 185 186 187 188 189]
  [190 191 192 193 194 195 196 197 198 199]]]


In [21]:
import numpy as np
a = [1,2,3]
b = a[1: 1]
b

[]

In [18]:
from tkinter import *

# pip install pillow
from PIL import Image, ImageTk
load = []
class Window(Frame):
    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.master = master
        self.pack(fill=BOTH, expand=1)
        
        load.append(ImageTk.PhotoImage(Image.open("img/parrot.png")))
        #render = ImageTk.PhotoImage(load[0])
        img = Label(self, image=load[0])
        img.image = load[0]
        img.place(x=0, y=0)

        
root = Toplevel()
app = Window(root)
root.wm_title("Tkinter window")
root.geometry("200x120")
root.mainloop()

In [11]:
# Python3 code here for creating class 
class geeks:  
    def __init__(self, x, y):  
        self.x = x  
        self.y = y 
          
    def Sum(self): 
        print( self.x + self.y ) 
   
# creating list        
list = []  
  
# appending instances to list  
list.append( geeks(2, 3) ) 
list.append( geeks(12, 13) ) 
list.append( geeks(22, 33) ) 
  
for obj in list: 
    # calling method  
    obj.Sum() 
  
# We can also access instances method 
# as list[0].Sum() , list[1].Sum() and so on. 

5
25
55


In [ ]:
# displsy how many win/ loss/ tie on UI
# put UI/ model into a seperate py file once have time

import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
import random
from shutil import copyfile
from tkinter import *

#self created lib in py file format
from Model.Game_Agent import game_agent
from Model.DL_Network_Model import Net
from Model.Funtion_Bank import data_input

file_name_model_latest_version = 'Model/model_latest_version.pt'
file_name_model_last_version = 'Model/model_last_version.pt'

global game_result
global tie
global win
global loss

def trigger_AI(data_original):
    game_result = "Ongoing.."
    torch_tensor = torch.tensor(data_original, dtype=torch.float)
    
    #another possible scenario: if input has worong occurance of 1 & -1, then it is not valid input as well - pending for coding
    game_a = game_agent(torch_tensor, 0)
    if game_a.verify_result() == True:
        print('Win')
        game_result = "Win"
        next_step = -1
    elif 0 not in torch_tensor:
        print('tie')
        game_result = "tie"
        next_step = -1
    else:
        #here it can be 3 levels of AI for selection
        if AI_difficulty == 'H':
            next_vision = model_latest_version(torch_tensor)
        else:
            next_vision = model_last_version(torch_tensor)
        #comprehance predict result to make impossible option as '-1'
        for k in range(0,9):
            if torch_tensor[k] != 0:
                next_vision[k] = -1.1
        next_step = next_vision.argmax()
        print(next_vision)
        torch_tensor[next_step] = 1
        game_a = game_agent(torch_tensor, 0)
        # varify status of result
        if game_a.verify_result() == True:
            game_result = 'Loss'
        elif 0 not in torch_tensor:
            print('tie')
            game_result = 'tie'
            #tie = tie + 1
    return next_step, game_result

def refresh_game_board(data_original):
    for i in range(0,9):
        if data_original[i] == -1:
            game_board[i] = 'o'
        elif data_original[i] == 1:
            game_board[i] = 'x'
        else:
            game_board[i] = ''
    btn_00_text.set(game_board[0])
    btn_01_text.set(game_board[1])
    btn_02_text.set(game_board[2])
    btn_10_text.set(game_board[3])
    btn_11_text.set(game_board[4])
    btn_12_text.set(game_board[5])
    btn_20_text.set(game_board[6])
    btn_21_text.set(game_board[7])
    btn_22_text.set(game_board[8])
    
def button_00_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_00_text.get() == '':
        btn_00_text.set("o")
        data_original[0] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)
        
def button_01_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_01_text.get() == '':
        btn_01_text.set("o")
        data_original[1] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)

def button_02_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_02_text.get() == '':
        btn_02_text.set("o")
        data_original[2] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)
        
def button_10_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_10_text.get() == '':
        btn_10_text.set("o")
        data_original[3] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)

def button_11_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_11_text.get() == '':
        btn_11_text.set("o")
        data_original[4] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)
        
def button_12_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_12_text.get() == '':
        btn_12_text.set("o")
        data_original[5] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)
        
def button_20_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_20_text.get() == '':
        btn_20_text.set("o")
        data_original[6] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)

def button_21_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_21_text.get() == '':
        btn_21_text.set("o")
        data_original[7] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)
        
def button_22_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_22_text.get() == '':
        btn_22_text.set("o")
        data_original[8] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)

def btn_reset_clicked():
    reset_game()
    refresh_game_board(data_original)
    
def model_select():
    global model_selection
    if AI_difficulty == 'L':
        model_selection = state_dict_last_version
    else:
        model_selection = file_name_model_latest_version

def reset_game():
    global data_original
    global game_board
    data_original = [0,0,0,0,0,0,0,0,0]
    game_board = ['','','','','','','','','']

    if (AI_first == 1) & (AI_random_first_step == 1):
        data_original[random.randint(0,8)] = 1
    elif (AI_first == 1):
        torch_tensor = torch.tensor(data_original, dtype=torch.float)
        if AI_difficulty == 'H':
            next_vision = model_latest_version(torch_tensor)
        else:
            next_vision = model_last_version(torch_tensor)
        #comprehance predict result to make impossible option as '-1.1'
        for k in range(0,9):
            if torch_tensor[k] != 0:
                next_vision[k] = -1.1
        next_step = next_vision.argmax()
        data_original[next_step] = 1
        
def AI_difficulty_selection():
    global AI_random_first_step
    global AI_difficulty
    if AI_difficulty_var.get() == 'H':
        AI_random_first_step = 0
        AI_difficulty = 'H'
    elif AI_difficulty_var.get() == 'M':
        AI_random_first_step = 1
        AI_difficulty = 'M'
    elif AI_difficulty_var.get() == 'L':
        AI_random_first_step = 1
        AI_difficulty = 'L'
    print(AI_difficulty_var.get())
    reset_game()
    refresh_game_board(data_original)

def Who_play_first_selection():
    global AI_first
    print(Who_play_first_var.get())
    if Who_play_first_var.get() == 'AI':
        AI_first = 1
    else:
        AI_first = 0
    reset_game()
    refresh_game_board(data_original)

if __name__ == "__main__":
    AI_first = 1
    #AI_random_first_step = 0
    tie = 0
    win = 0
    loss = 0

    #load model
    model_last_version = Net()
    state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
    model_last_version.load_state_dict(state_dict_last_version)
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)

    #Load game UI:
    window = Tk()
    window.title("Welcome to Game made by Hongxu")
    window.geometry('450x200')
    lbl = Label(window, text="Ready to start")
    
    btn_00_text = StringVar()
    btn_01_text = StringVar()
    btn_02_text = StringVar()
    btn_10_text = StringVar()
    btn_11_text = StringVar()
    btn_12_text = StringVar()
    btn_20_text = StringVar()
    btn_21_text = StringVar()
    btn_22_text = StringVar()

    btn_00 = Button(window, textvariable=btn_00_text, command=button_00_clicked, height = 2, width = 4)
    btn_00.grid(row=0, column=0)
    btn_01 = Button(window, textvariable=btn_01_text, command=button_01_clicked, height = 2, width = 4)
    btn_01.grid(row=0, column=1)
    btn_02 = Button(window, textvariable=btn_02_text, command=button_02_clicked, height = 2, width = 4)
    btn_02.grid(row=0, column=2)
    btn_10 = Button(window, textvariable=btn_10_text, command=button_10_clicked, height = 2, width = 4)
    btn_10.grid(row=1, column=0)
    btn_11 = Button(window, textvariable=btn_11_text, command=button_11_clicked, height = 2, width = 4)
    btn_11.grid(row=1, column=1)
    btn_12 = Button(window, textvariable=btn_12_text, command=button_12_clicked, height = 2, width = 4)
    btn_12.grid(row=1, column=2)
    btn_20 = Button(window, textvariable=btn_20_text, command=button_20_clicked, height = 2, width = 4)
    btn_20.grid(row=2, column=0)
    btn_21 = Button(window, textvariable=btn_21_text, command=button_21_clicked, height = 2, width = 4)
    btn_21.grid(row=2, column=1)
    btn_22 = Button(window, textvariable=btn_22_text, command=button_22_clicked, height = 2, width = 4)
    btn_22.grid(row=2, column=2)
    lbl.grid(row=4, column=3)
    
    AI_difficulty_lbl = Label(window, text="AI difficulty:")
    AI_difficulty_lbl.grid(row=1, column=3)
    AI_difficulty_var = StringVar()
    AI_difficulty_r1 = Radiobutton(window, text='High', variable=AI_difficulty_var, value='H', justify=LEFT, command=AI_difficulty_selection)
    AI_difficulty_r2 = Radiobutton(window, text='Medium', variable=AI_difficulty_var, value='M', justify=LEFT, command=AI_difficulty_selection)
    AI_difficulty_r3 = Radiobutton(window, text='Low', variable=AI_difficulty_var, value='L', justify=LEFT, command=AI_difficulty_selection)
    AI_difficulty_var.set("H")
    AI_difficulty_selection()
    AI_difficulty_r1.grid(row=1, column=5)
    AI_difficulty_r2.grid(row=1, column=6)
    AI_difficulty_r3.grid(row=1, column=7)
    
    Who_play_first_lbl = Label(window, text="Who play first?")
    Who_play_first_lbl.grid(row=0, column=3)
    Who_play_first_var = StringVar()
    Who_play_first_r1 = Radiobutton(window, text='Player', variable=Who_play_first_var, value='Player', justify=LEFT, command=Who_play_first_selection)
    Who_play_first_r2 = Radiobutton(window, text='AI', variable=Who_play_first_var, value='AI', justify=LEFT, command=Who_play_first_selection)
    Who_play_first_var.set("Player")
    Who_play_first_r1.grid(row=0, column=5)
    Who_play_first_r2.grid(row=0, column=6)
    Who_play_first_selection()
    
    #button to restart
    btn_reset = Button(window, text='Restart', command=btn_reset_clicked, justify=LEFT, height = 2, width = 8)
    btn_reset.grid(row=2, column=5)

    reset_game()
    refresh_game_board(data_original)

    window.mainloop()

In [39]:
import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
import random
from shutil import copyfile
from tkinter import *


def button_refresh():
    #lbl.configure(text="Button was clicked !!")
    for i in range(0, rows):
        for j in range(0, columns):
            #btn_txt_matrix.append(StringVar())
            btn_txt_matrix[count].set(" ")
    """
    if btn_20_text.get() == '':
        btn_20_text.set("o")
        data_original[6] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        if next_step != -1:
            data_original[next_step] = 1
        refresh_game_board(data_original)
        lbl.configure(text= game_result)
    """
        
if __name__ == "__main__":
    playground_h = 12
    playground_w = 10
    rows = playground_h
    columns = playground_w * 3 + 2
    btn_matrix = []
    btn_matrix = []
    btn_refresh_matrix = []
    btn_txt_matrix = []
    
    #load mode
    
    #Load game UI:
    window = Tk()
    window.title("Welcome to Game made by Hongxu")
    window.geometry('1400x600')
    lbl = Label(window, text="Ready to start")
    
    count = 0
    for i in range(0, rows):
        for j in range(0, columns):
            btn_txt_matrix.append(StringVar())
            btn_txt_matrix[count].set("o")
            btn_refresh_matrix.append(button_refresh())
            btn_matrix.append(Button(window, textvariable = btn_txt_matrix[count], command = btn_refresh_matrix[count], height = 2, width = 4))
            btn_matrix[count].grid(row = i, column = j)
            count += 1
    window.mainloop()